# Deproject IM Lup

Download DSHARP image

    !wget https://almascience.eso.org/almadata/lp/DSHARP/images/IMLup_continuum.fits

The sphere image needs to be requested by the authors.

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import griddata

from gofish import imagecube

figpath = Path('figures')
figpath.mkdir(exist_ok=True)

## Select either ALMA or SPHERE by running that cell

### ALMA

In [ ]:
fname = 'IMLup_continuum.fits'
name = 'IM Lup (ALMA)'
clip = 3.0

opts = dict(inc=47.5, PA=144.55, z0=0.0, psi=0.0, q_taper=1.0, r_taper=np.inf)

### SPHERE

\begin{align}
\psi &= 1.27\\
h/r &= 0.18\text{ at 100 au}
\end{align}

In [ ]:
fname = 'IM_Lup_reducedRob_median_Hband_12.25mas_mod.fits'
name = 'IM Lup (SPHERE)'
clip = 4.0
dpc = 155.82
z0 = -18 * (dpc / 100)**1.271 / dpc

opts = dict(PA=325., inc=54.75, z0=z0, psi=1.271, q_taper=1.0, r_taper=np.inf)

## Define the surface shape

$$z(r) = z_0 \times \left(\frac{r}{1^{\prime\prime}}\right)^{\psi}  \times \exp\left( -\left[\frac{r}{r_{\rm taper}} \right]^{q_{\rm taper}} \right)$$

In [ ]:
def z_fct(ras):
    return opts['z0'] * ras**opts['psi'] * np.exp(-(ras / opts['r_taper'])**opts['q_taper'])

## Process data

### Read data

In [ ]:
img = imagecube(fname, FOV=clip)

### Compute sky coordinates

In [ ]:
r, th, z = img.disk_coords(z_func=z_fct, **opts)
#r, th, z = img.disk_coords(**opts)

### Deprojection

Compute regular grids to interpolate on

In [ ]:
rui = np.linspace(0, 1.5, 101)
tui = np.linspace(-np.pi, np.pi, 111)

ru = 0.5 * (rui[1:] + rui[:-1])
tu = 0.5 * (tui[1:] + tui[:-1])

RUI, TUI = np.meshgrid(rui, tui, indexing='ij')
RU, TU = np.meshgrid(ru, tu, indexing='ij')

XUI = RUI * np.cos(TUI)
YUI = RUI * np.sin(TUI)

Interpolate = deproject

In [ ]:
new_points = np.vstack((RU.ravel(), TU.ravel())).T
points = np.vstack((r.ravel(), th.ravel())).T
values = img.data.ravel()

In [ ]:
im_dp = griddata(points, values, (RU, TU), method='linear')

## Plotting

In [ ]:
norm = LogNorm(1e-3 * img.data.max(), img.data.max())
figname = name.replace('(','').replace(')','').replace(' ','_')

### Projected surface and deprojected image

In [ ]:
r_circles = [0.5, 1, 1.5, 2]

f, axs = plt.subplots(1, 2, figsize=(10, 5))

ax = axs[0]
ax.pcolormesh(img.xaxis, img.yaxis, img.data, norm=norm)
ax.set_xlim(ax.get_xlim()[::-1])
ax.set_aspect(1)
ax.text(0.015, 0.95, name, transform=axs[0].transAxes, color='w', bbox={'color':'k','alpha':0.45})

cc_r = ax.contour(img.xaxis, img.yaxis, r, r_circles, linewidths=1, linestyles='--', colors='w')
ax.clabel(cc_r)
cc_t = ax.contour(img.xaxis, img.yaxis, np.rad2deg(th+np.pi), np.arange(0, 360, 45) ,linewidths=1, linestyles='--', colors='k')
ax.clabel(cc_t)

ax = axs[1]
ax.pcolormesh(XUI, YUI, im_dp, norm=norm)
ax.set_aspect(1)
for _r in r_circles:
    c = plt.Circle((0, 0), radius=_r, fc='none', ec='w', ls='--')
    ax.text(-_r, 0, f'{_r:.1f}"', rotation=90, c='w', ha='right', va='center')
    ax.add_artist(c)
    
f.savefig(figpath / f"{figname}_deprojection.png")

### Radial profile and residuals

Substracting azimuthally-averaged profile

In [ ]:
profile = im_dp.mean(-1)

f, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].semilogy(ru, profile)
axs[0].set_title('radial profile')

ax = axs[1]
cc = ax.pcolormesh(XUI, YUI, im_dp / profile[:, None] - 1)
ax.set_aspect(1)
cb = f.colorbar(cc, ax=ax, orientation='horizontal', fraction=0.05, location='top', pad=0)
cb.set_label('fractional residual')
for _r in r_circles:
    c = plt.Circle((0, 0), radius=_r, fc='none', ec='w', ls='--')
    ax.text(-_r, 0, f'{_r:.1f}"', rotation=90, c='w', ha='right', va='center')
    ax.add_artist(c)

f.savefig(figpath / f"{figname}_profile.png")

phase function as function of azimuth (note: $\neq$ scattering angle!)

In [ ]:
plt.plot(tu, np.nanmean(im_dp / profile[:, None] - 1, 0));